OpenAI Finetuning Proof of Concept

In [8]:
!pip install openai==0.28 datasets

In [9]:
from openai import FineTuningJob, ChatCompletion
from datasets import load_dataset 
from time import sleep
import random 
import json

In [10]:
yahoo_answers_qa = load_dataset("yahoo_answers_qa", split="train")
yahoo_answers_qa

Dataset({
    features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
    num_rows: 87362
})

In [16]:
# helper function to format data for fine-tuning
def format_data(data, prompt="You are a butler who answers questions. Answer any question politely."):
    
    formatted_data = [{
            "messages": [
                {"role": "system", "content": prompt},
                {"role": "user", "content": message["question"]},
                {"role": "assistant", "content": message["answer"]}
            ] 
        } for message in data 
    ]
    
    random.shuffle(formatted_data)
    
    return formatted_data

In [17]:
formatted_data = format_data(yahoo_answers_qa)
formatted_data[0]

{'messages': [{'role': 'system',
   'content': 'You are a butler who answers questions. Answer any question politely.'},
  {'role': 'user',
   'content': 'How to banish unwanted Angelic presences from my area?'},
  {'role': 'assistant',
   'content': 'I know what you mean, if you were to actually believe the Bible, no one would want to be in the presents of an Angel. In the Bible it was very rare for an Angel to be on Earth for any good purpose, most of the time they are there to deal out punishment.'}]}

In [29]:
# standard train test split
train_size = int(0.8 * len(formatted_data))
train_data = formatted_data[:train_size]
test_data = formatted_data[train_size:]

In [32]:
def save_data(dictionary_data, file_name):
    with open(file_name, 'w') as f:
        for entry in dictionary_data:
            json.dump(entry, f)
            f.write('\n')

In [33]:
save_data(train_data, "train.jsonl")
save_data(test_data, "test.jsonl")

In [34]:
import os
import openai

In [47]:
# upload data to OpenAI to prepare for fine tuning

def upload_fine_tuning_data(file_name):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    with open(file_name, 'rb') as file:
        return openai.files.create(file=file, purpose="fine-tune")

In [48]:
uploaded_training_data = upload_fine_tuning_data("train.jsonl")
uploaded_training_id = uploaded_training_data.id

In [57]:
uploaded_validation_data = upload_fine_tuning_data("test.jsonl")
uploaded_validation_id = uploaded_validation_data.id

In [81]:
def fine_tune_model(uploaded_training_id, uploaded_validation_id, model="davinci"):
    openai.api_key = os.getenv("OPENAI_API_KEY")

    try:
        return openai.fine_tuning.jobs.create(
            training_file=uploaded_training_id,
            model=model,
        )
    except openai.error.OpenAIError as e:
        print(e)

In [82]:
openai_model = "gpt-3.5-turbo"
fine_tuning_response = fine_tune_model(uploaded_training_id, uploaded_validation_id, model=openai_model)

In [83]:
fine_tuning_id = fine_tuning_response.id
fine_tuning_id

'ftjob-JQ4mexJHjEL4YybfZBkZeZPu'

In [89]:
# monitor fine-tuning status

import time
while True:
    fine_tuning_response = openai.fine_tuning.jobs.retrieve(fine_tuning_id)
    print(fine_tuning_response.status)

    if fine_tuning_response.status == "succeeded":
        break
    else:
        time_now = time.strftime("%H:%M:%S", time.localtime())
        print(f"{time_now} Waiting for fine tuning to complete...")
        sleep(30)

validating_files
01:30:40 Waiting for fine tuning to complete...
validating_files
01:31:10 Waiting for fine tuning to complete...
validating_files
01:31:40 Waiting for fine tuning to complete...
validating_files
01:32:11 Waiting for fine tuning to complete...
validating_files
01:32:41 Waiting for fine tuning to complete...
validating_files
01:33:11 Waiting for fine tuning to complete...
